# TP1

Autores: Afonso FFerreira pg52669, Tiago Rodrigues pg52705

### Exercício 1

Use a package Cryptography   e  o package ascon para  criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:

- a) Autenticação do criptograma e dos metadados (associated data) usando Ascon em modo de cifra.

- b) As chaves de cifra, autenticação  e  os “nounces” são gerados por um gerador pseudo aleatório (PRG)  usando o Ascon em modo XOF. As diferentes chaves para inicialização do PRG são inputs do emissor e do receptor.

- c) Para implementar a comunicação cliente-servidor use o package python asyncio

In [8]:
from ascon import encrypt

import os

key = os.urandom(16)

nonce = os.urandom(16)

plaintext = b"Anacleto manda mensagem a Bernardina"

associateddata = b''


criptograma = encrypt(key, nonce, associateddata, plaintext, variant="Ascon-128")
criptograma

b'B3\xdcv\xfa\x9a0\x14\xad$F\x1f{\x134u\xf2\x7f\x9d\x00\n\t\xef5j\x9f\x89JG-^_\\\xf8\xec\xbe\xb0\x85H\xac\xa2\x05\xcd\xc5\xf502\xa5\xcdI\x9d\x8e'